# Classification after Clustering with Agglomerative Clustering

### Import CSV and Required Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.pandas.set_option('display.max_columns', None)
df = pd.read_csv(r'./data/clustered_data.csv')
print(df.shape)

(2240, 22)


**Split X and y**
- Why do we split our data?
> Training Dataset is the part of Original Dataset that we use to train our ML model. The model learns on this data by running the algorithm and maps a function F(x) where “x” in the independent variable (inputs) for “y” where “y” is the dependent variable(output).

In [3]:
X = df.drop('cluster', axis = 1)
y = df['cluster']

In [4]:
X

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth
0,65,2,0,0,0,58138.0,1617.0,4850.0,58,635,81,546,120.5,81,88.0,8,10,4,3,0,7
1,68,2,0,1,2,46344.0,27.0,4300.0,38,11,1,6,2.0,1,6.0,1,1,2,2,0,5
2,57,2,1,0,0,71613.0,776.0,4499.0,26,426,49,127,111.0,21,42.0,8,2,10,1,0,4
3,38,2,1,1,1,26646.0,53.0,4326.0,26,11,4,20,10.0,3,5.0,2,0,4,2,0,6
4,41,4,1,1,1,58293.0,422.0,4348.0,94,173,43,118,46.0,27,15.0,5,3,6,5,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,55,2,1,1,1,61223.0,1341.0,4568.0,46,709,43,182,42.0,81,126.5,9,3,4,2,0,5
2236,76,4,1,1,3,64014.0,444.0,4206.0,56,406,0,30,0.0,0,8.0,8,2,5,7,1,7
2237,41,2,0,0,0,56981.0,1241.0,4342.0,91,908,48,217,32.0,12,24.0,2,3,13,1,1,6
2238,66,3,1,1,1,69245.0,843.0,4343.0,8,428,30,214,80.0,30,61.0,6,5,10,2,0,3


In [5]:
y

0       0
1       1
2       0
3       1
4       2
       ..
2235    2
2236    2
2237    0
2238    0
2239    1
Name: cluster, Length: 2240, dtype: int64

## Grid Search

- Why do we use Grid Search?

`GridSearchCV` is a technique to search through the best parameter values from the given set of the grid of parameters. It is basically a cross-validation method. the model and the parameters are required to be fed in. Best parameter values are extracted and then the predictions are made.

## Select the best model
- so here we have some list of the best classification algorithms we imported. Now we will compare each model's score and see which model is performing better than rest of the others

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score,roc_curve,confusion_matrix

models = {
    "Support Vector Classifier" : SVC(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression(),
    "K-Neighbors Classifier": KNeighborsClassifier(),
    "XGBClassifier": XGBClassifier(), 
    # "CatBoosting Classifier": CatBoostClassifier(verbose = False),
    "AdaBoost Classifier": AdaBoostClassifier()
}

In [7]:
models.values()

dict_values([SVC(), RandomForestClassifier(), DecisionTreeClassifier(), GradientBoostingClassifier(), LogisticRegression(), KNeighborsClassifier(), XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...), AdaBoostClassifier()])

In [8]:
# Create a function which can evaluate models and return a report
def evaluate_models(X, y, models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics with cost
    '''
    
    # Separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    
    model_list = []
    scores = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) # Train model
        
        # Make Predictions
        y_pred = model.predict(X_test)
        
        score = accuracy_score(y_test, y_pred)
        
        model_name = list(models.keys())[i]
        
        print(f'---- Score for --- {model_name} ----')
        print(f'{score}')
        model_list.append(model_name)
        scores.append(score)
    
    print()
    
    report = pd.DataFrame()
    report['Model Name'] = model_list
    report['Score'] = scores
    
    return report

### Let's check the report

In [9]:
report = evaluate_models(X, y, models)

---- Score for --- Support Vector Classifier ----
0.7299107142857143
---- Score for --- Random Forest ----
0.9620535714285714
---- Score for --- Decision Tree ----
0.9419642857142857
---- Score for --- Gradient Boosting ----
0.9642857142857143
---- Score for --- Logistic Regression ----
0.8772321428571429
---- Score for --- K-Neighbors Classifier ----
0.8102678571428571
---- Score for --- XGBClassifier ----
0.9665178571428571
---- Score for --- AdaBoost Classifier ----
0.9441964285714286



In [10]:
report.sort_values('Score')

,Model Name,Score
0,Support Vector Classifier,0.729911
5,K-Neighbors Classifier,0.810268
4,Logistic Regression,0.877232
2,Decision Tree,0.941964
7,AdaBoost Classifier,0.944196
1,Random Forest,0.962054
3,Gradient Boosting,0.964286
6,XGBClassifier,0.966518


- ### From the report above we can see that the Logistic Regression Classifier model performed the best, so we will continue training our model using Logistic Regression Classifier algorithm.

### Split into Train and test data

- **Do you know why we split the train and test dataset?**
> The train test split technique can be used for classification and regression problems to test machine learning algorithms. The procedure takes the given dataset and splits it into two subsets: ```Training data/train set:``` it is used to train the algorithm and fit the machine learning model
then we have ```test data/test set``` which is basically a different data for which we know the values but this data was never shown to the model before. Thus if the model after training is performing good on test set as well then we can say that the Machine Learning model is good.

In [11]:
# Separate data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
X_test

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth
324,69,2,0,1,1,40464.0,630.0,4721.0,78,424,17,118,7.0,23,41.0,8,2,8,6,0,8
96,62,2,0,1,1,47916.0,606.0,4771.0,72,505,0,26,0.0,0,75.0,7,4,6,5,1,6
2104,50,0,1,0,0,14188.0,75.0,4673.0,40,2,7,11,16.0,12,27.0,2,0,4,1,0,6
1259,53,2,1,0,0,76653.0,2279.0,4504.0,91,736,63,556,120.5,81,126.0,4,7,11,1,2,2
1061,64,2,1,1,2,65196.0,1155.0,4526.0,34,743,19,181,12.0,0,126.5,7,6,11,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,48,3,1,0,0,82584.0,1435.0,4577.0,26,1076,68,103,29.0,81,68.0,3,4,8,1,2,1
1340,60,2,1,1,1,50127.0,320.0,4218.0,88,274,0,21,4.0,6,15.0,5,1,6,1,1,6
755,44,1,1,0,0,63855.0,1006.0,4692.0,28,359,35,314,93.0,81,89.0,7,3,13,1,0,4
2138,68,3,0,1,1,62637.0,185.0,4307.0,76,104,12,48,4.0,12,5.0,2,2,5,1,0,2


### Let's do hyperparameter tuning
- **And what's it actually?**

> A Machine Learning model is defined as a mathematical model with a number of parameters that need to be learned from the data. By training a model with existing data, we are able to fit the model parameters. 
However, there is another kind of parameter, known as Hyperparameters, that cannot be directly learned from the regular training process. They are usually fixed before the actual training process begins. These parameters express important properties of the model such as its complexity or how fast it should learn. 

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': np.random.randint(100,500, 10),
    'multi_class': ['auto', 'ovr', 'multinomial'],
    "C":np.logspace(-3,3,7),
    "penalty":["l1","l2"]

    
}

logreg=LogisticRegression()

logreg_cv=GridSearchCV(logreg,params,cv=10)
logreg_cv.fit(X_train,y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)